In [9]:
max_rounds = 400
optimize_rounds = 0.07
learning_rate=0.07
early_stopping_rounds=50



it is recommended initially setting max_rounds fairly high and using optimize_rounds to get an idea of the appropriate umber of rounds(should be close to the maximum value of best_ntree_limit among all folds, maybe even a bi higher if your model is adequately regularized. or you could set verbose=True and look at the details to try to find a number of rounds that works well for all folds.) then you would turn off optimize_rounds and set max_rounds to the appropriate number of total rounds.

In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from numba import jit
import gc
import time 

In [10]:
#compute gini

@jit
def eval_gini(y_true, y_prob) :
  y_true = np.asarray(y_true)
  y_true = y_true[np.argsort(y_prob)]
  ntrue=0
  gini=0
  delta=0
  n=len(y_true)
  for i in range(n-1,-1,-1) :
    y_i=y_true[i]
    ntrue+=y_i
    gini+=y_i*delta
    delta+=1-y_i
  gini = 1-2*gini / (ntrue*(n-ntrue))
  return gini

In [11]:
#function from olivier's kernel

def gini_xgb (preds, dtrain) :
  labels = dtrain.get_label()
  gini_score = -eval_gini(labels, preds)
  return [('gini', gini_score)]

def add_noise(series, noise_level) :
  return series * (1+noise_level * np.random.randn(len(series)))

def target_encode(trs_series=None, tst_series=None, target=None, min_samples_leaf=1, smoothing=1, noise_level=0):
  assert len(trn_series) == len(target)
  assert trn_series.name == tst_series.name
  temp=pd.concat([trn_series, target], axis=1)

  averages = temp.groupby(trn_series.name)[target.name].agg(['mean','count'])

  smoothing = 1/ (1+np.exp(-(averages['count'] - min_samples_leaf) / smoothing))

  prior = target.mean()

  averages[target.name] = prior * (1-smoothing) + averages['mean'] * smoothing

  averages.drop(['mean','count'],axis=1, inplace=True)
#################################################################
  ft_trn_series = pd.merge(
      trn.series.to_frame(trn_series.name),
      averages.reset_index().rename({'index' : target.name, target.name : 'average'}),
      on=trn_series.name,
      how='left')['average'].rename(trn_series.name + 'mean').fillna(prior)

  ft_trn_series.index=trn_series.index

  ft_val_series = pd.merge(
      val.series.to_frame(val_series.name),
      averages.reset_index().rename({'index' : target.name, target.name : 'average'}),
      on=val_series.name,
      how='left')['average'].rename(val_series.name + 'mean').fillna(prior)

  ft_val_series.index=val_series.index

  ft_tst_series = pd.merge(
      tst.series.to_frame(tst_series.name),
      averages.reset_index().rename({'index' : target.name, target.name : 'average'}),
      on=tst_series.name,
      how='left')['average'].rename(tst_series.name + 'mean').fillna(prior)

  ft_tst_series.index=tst_series.index

  return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)



In [ ]:
#read data

# train_df = pd.read_csv('../input/train.csv', na_values='-1')

# test_df=pd.read_csv('../input/test.csv', na_values='-1')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
direc='/content/drive/MyDrive/주피터_대피소/kaggle&github/2. Driver Prediction/'

train_df=pd.read_csv(direc+'train.csv', na_values='-1')
test_df=pd.read_csv(direc+'test.csv', na_values='-1')

In [12]:
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
  "ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
#add combinations
combs= [
    ('ps_reg_01', 'ps_car_02_cat'),
    ('ps_reg_01', 'ps_car_04_cat')
]

In [13]:
#process data

id_test=test_df["id"].values
id_train =train_df['id'].values
y=train_df['target']


start=time.time()

for n_c , (f1,f2) in enumerate(combs) :
  name1 = f1 + '_plus_' + f2

  print('current feature %60s %4d in %5.1f' % (name1, n_c+1, (time.time() - start)/60), end='')
  print('\r'*75, end='')

  train_df[name1] = train_df[f1].apply(lambda x : str(x)) + '_' + train_df[f2].apply(lambda x : str(x))
  test_df[name1] = test_df[f1].apply(lambda x : str(x)) + '_' + train_df[f2].apply(lambda x  : str(x))

  lbl = LabelEncoder()
  lbl.fit((list(train_df[name1].values)) + list(test_df[name1].values))

  train_df[name1] = lbl.transform(list(train_df[name1].values))
  test_df[name1] = lbl.transform(list(test_df[name1].values))

  train_features.append(name1)

x=train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in x.columns if '_cat' in f]

KeyError: ignored

In [ ]:
y_valid_pred = 0*y
y_test_pred = 0

In [ ]:
#set up folds

k=5
kf = KFold(n_splits=k, random_state=42, shuffle = True)
np.random.seed(0)

In [ ]:
# set up classifier

model = XGBClassifier(
    n_estimators=max_rounds,
    max_depth=4,
    objective = 'binary:logistic',
    learning_rate = learning_rate,
    subsample=.8,
    min_dhild_weight=6,
    colsample_bytree=.8,
    scale_pos_weight=1.6,
    gamma=10,
    reg_alpha=8,
    reg_lambda=1.3
)

In [ ]:
#run cv

for i, (train_index, test_index) in enumerate(kf.split(train_df)) :
  #create data for this fold


  y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
  x_train, x_valid = x.iloc[train_index, :].copy(), x.iloc[test_index, :].copy()

  x_test = test_df.copy()

  print('\nFold', i)

  #encode data
  for f in f_cats :
    x_train[f+'_avg'], x_valid[f+'_avg'], x_test[f+'_avg'] = target_encode(trn_series=x_train[f], 
                                                                           val_series=x_valid[f],
                                                                           tst_series=x_test[f],
                                                                           target=y_train,
                                                                           min_samples_leaf=200,
                                                                           smoothing=10,
                                                                           noise_level=0)
    #run model for this fold
    if optimize_rounds :
      eval_set=[(x_valid, y_valid)]
      fit_model = model.fit(x_train, y_train, eval_set=eval_set,
                            eval_metric=gini_xgb,
                            early_stopping_rounds=early_stopping_rounds,
                            verbose=False)
      print('Best N trees : ', model.best_ntree_limit)
      print('Best gini : ', model.best_score_)
    else :
      fit_model = model.fit(x_train,y_train)

    
    #generate validation predictions for this fold
    pred = fit_model.predict(x_valid)[:,1]
    print('gini :', eval_gini(y_valid, pred))
    y_valid_pred.iloc[test_index] = pred

    #Accumulate test set predictions
    y_test_pred += fit_model.predict(x_test)[:,1]

    del x_test, x_train, x_valid, y_train

  
y_test_pred /= k      #average test set predictions


print('\ngini for full training set:')
eval_gini(y,y_valid_pred)

In [ ]:
# save validation predictions for stacking/ensembling

val=pd.DataFrame()

val['id'] = id_train
val['target'] = y_valid_pred.values

val.to_csv('xgv_valid.csv', float_format='%.6f', index=False)

In [ ]:
#Create submission file
sub=pd.DataFrame()
sub['id']=id_test
sub['target']=y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)